In [154]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score
from keras.utils import to_categorical
from sklearn.preprocessing import OrdinalEncoder

%matplotlib inline
import matplotlib.pyplot as plt


In [2]:
data_exp=pd.read_csv('BikeSharing_Bluebikes2022.csv')
nat_hol = ['2022-01-01','2022-01-17','2022-02-21','2022-04-17','2022-04-18','2022-05-08','2022-05-30','2022-06-19','2022-06-20','2022-07-04']

data_exp['is_Holiday'] = [ 1 if data_exp.iloc[i]['starttime'][0:10] in nat_hol else 0 for i in range(len(data_exp))]
data_w = pd.read_csv("Boston 2022-01-01 to 2022-08-31.csv")
drop = ['dew', 'sunrise','sunset','moonphase','conditions', 'description','stations','sealevelpressure',
'windgust','severerisk', 'uvindex', 'solarenergy', 'solarradiation','preciptype','winddir','name']
data_w = data_w.drop(columns = drop)
data_w['datetime'] = pd.to_datetime(data_w['datetime'])

In [41]:
from datetime import datetime as dt

data_exp['starttime'] = pd.to_datetime(data_exp['starttime'], format='%Y-%m-%d %H:%M:%S')
data_exp['datetime'] = pd.to_datetime(data_exp['starttime']).dt.date
data_exp['datetime'] = pd.to_datetime(data_exp['datetime'])

data_merge = data_exp.merge(data_w, how='left', on='datetime')

In [42]:
data_merge.columns
data_merge = data_merge.drop(columns=(['Unnamed: 0']))

In [43]:
data_merge.columns


Index(['tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'postal code', 'is_Holiday', 'datetime', 'tempmax', 'tempmin', 'temp',
       'feelslikemax', 'feelslikemin', 'feelslike', 'humidity', 'precip',
       'precipprob', 'precipcover', 'snow', 'snowdepth', 'windspeed',
       'cloudcover', 'visibility', 'icon'],
      dtype='object')

In [44]:
data_merge['start station id'].unique()

array([178, 189,  94,  19, 107,  36,  58,  60,   4, 377, 342, 491, 362,
        43, 121, 381,  10,  81, 332, 328, 233, 176,  55,  80,  12,  68,
        32, 202,  54,  74,  39,  66, 356, 116, 398,  42, 352, 330, 426,
        67, 225, 234,  91,  76, 161, 386, 374, 345, 437, 487,  40, 415,
       149, 183,  23,  71,   6,  72, 131, 118,  87,  86,  25,  97, 119,
       338,  69, 359, 446, 239,  22,  47,  26, 125,  33, 157, 105, 471,
        41, 174,  30,  16, 400, 177, 191, 152, 378, 218, 179, 403,  75,
       443, 379,  17,  46,   7,  11, 226, 363, 479, 190, 536, 141,  78,
       181, 139, 380, 417,  95,  90, 364, 549, 529,  44, 151, 376, 100,
        24, 137, 104, 389,  20,  96, 108,  98, 272, 187,  21, 541, 370,
        59, 393, 163, 413,  65, 215, 175, 414, 186, 318,  93, 344, 554,
       472,  63, 169, 196,  51, 419, 331,  84, 478, 515, 115, 213, 372,
       361, 192,  31, 136, 197, 138, 110, 273,  37,  49, 228, 456,  73,
       206,  70, 407, 194, 193, 440,   9, 480, 102, 327,   8,   

In [56]:
list = []
for i in data_merge['start station id'].unique():
    count = data_merge[data_merge['start station id']==i].count()[0]
    list.append(count)

maxim = data_merge['start station id'].unique()[list.index(max(list))]
print(maxim)
data_merge[data_merge['start station id']==maxim].count()[0]



67


49127

In [58]:
data_merge.columns

Index(['tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'postal code', 'is_Holiday', 'datetime', 'tempmax', 'tempmin', 'temp',
       'feelslikemax', 'feelslikemin', 'feelslike', 'humidity', 'precip',
       'precipprob', 'precipcover', 'snow', 'snowdepth', 'windspeed',
       'cloudcover', 'visibility', 'icon'],
      dtype='object')

In [108]:
data_merge = data_merge[data_merge['start station id']==maxim]
data_nn = data_merge.drop(columns=['start station id',
       'start station name', 'start station latitude',
       'start station longitude','end station name',
       'end station latitude', 'end station longitude','bikeid','postal code', 'stoptime'])

In [91]:
ran = data_nn['starttime'].iloc[3]
ran.hour
(data_nn['starttime'].iloc[0].hour >= 0) & (data_nn['starttime'].iloc[0].hour < 6)

True

In [107]:
range(len(data_nn))
# data_nn['tripduration'].iloc[0]=2
# data_nn['tripduration'].iloc[0]

C:\Users\mirun\AppData\Local\Temp\ipykernel_18588\3059181073.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_nn['tripduration'].iloc[0]=2


2

In [113]:
data_nn['time_of_day'] = 0
for i in range(len(data_nn)):
    if (data_nn['starttime'].iloc[i].hour >= 6) & (data_nn['starttime'].iloc[i].hour < 12):
        data_nn['time_of_day'].iloc[i] = 'morning'
    elif (data_nn['starttime'].iloc[i].hour >= 12) & (data_nn['starttime'].iloc[i].hour < 18):
        data_nn['time_of_day'].iloc[i] = 'afternoon'
    elif (data_nn['starttime'].iloc[i].hour >= 18) & (data_nn['starttime'].iloc[i].hour < 24):
        data_nn['time_of_day'].iloc[i] = 'evening'
    elif (data_nn['starttime'].iloc[i].hour >= 0) & (data_nn['starttime'].iloc[i].hour < 6):
        data_nn['time_of_day'].iloc[i] = 'night'


C:\Users\mirun\AppData\Local\Temp\ipykernel_18588\3777789447.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_nn['time_of_day'].iloc[i] = 'night'


In [118]:
data_nn = data_nn.drop(columns=['starttime','datetime'])

Index(['tripduration', 'end station id', 'usertype', 'is_Holiday', 'tempmax',
       'tempmin', 'temp', 'feelslikemax', 'feelslikemin', 'feelslike',
       'humidity', 'precip', 'precipprob', 'precipcover', 'snow', 'snowdepth',
       'windspeed', 'cloudcover', 'visibility', 'icon', 'time_of_day'],
      dtype='object')

In [ ]:
data_nn.columns

In [120]:
data_nn_dummies = pd.get_dummies(data_nn, columns=['time_of_day', 'icon', 'usertype'])

In [165]:
data_nn_dummies_std = data_nn_dummies.copy()

In [166]:
col_std = ['tripduration', 'tempmax',
       'tempmin', 'temp', 'feelslikemax', 'feelslikemin', 'feelslike',
       'humidity', 'precip', 'precipprob', 'precipcover', 'snow', 'snowdepth',
       'windspeed', 'cloudcover', 'visibility']

data_nn_dummies_std[col_std]= (data_nn_dummies_std[col_std] - data_nn_dummies_std[col_std].mean()) / data_nn_dummies_std[col_std].std()

In [169]:
target = data_nn_dummies_std['end station id']
data_nn_dummies_std.set_index('end station id')
target.unique()

array([ 67, 139,  55,  36, 471, 108,  68, 178,  60,  75, 107, 372, 224,
        46, 115,  97,  91, 221, 377, 370,  11, 544, 515, 386,  76, 342,
        74, 352, 184,  44,  72, 105, 380, 361, 189,  98,  80, 185, 141,
       179,  95,  33,  84, 180,  89,   9, 408,  99, 177,  90, 335,  21,
        25, 338,  41, 413, 472,  54,  32, 381, 437, 549, 228,  42, 462,
       479,   6,  10, 190, 330, 374, 318,  16,  22, 364, 100,  39, 400,
       404, 116, 160, 193, 176, 417,  73, 110, 378, 399,   3, 143, 403,
       104, 225,  19,  12, 446,  86,  51, 233,  94, 332, 401,  66, 379,
       206,  81, 226, 554,  87, 333, 376, 398, 205, 131, 499,  37, 426,
       117, 553,  70, 345,  78, 234,  14, 195, 138,  43, 144,  49, 157,
       182, 331,  17,  77, 495,  40,  96, 133, 334, 111, 360, 102, 222,
        59,  58, 456,  69,  56,  15, 272, 181,  93, 175, 557, 492,  29,
       187,  47, 467,   4, 356, 359, 282, 344, 109, 530, 371,   8, 149,
        71, 536, 150,  27, 385, 452, 279, 463,  30, 367, 161, 41

In [168]:
target = to_categorical(target, num_classes=337)
target.shape

IndexError: index 471 is out of bounds for axis 1 with size 337

In [156]:
split = int(len(data_nn_dummies_std)*0.6)
split_val = int(len(data_nn_dummies_std)*0.7)
x_train = data_nn_dummies_std[:split]
y_train = target[:split]
y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
x_val = data_nn_dummies_std[split: split_val]
y_val = target[split: split_val]
y_val = np.asarray(y_val).astype('float32').reshape((-1,1))

x_test = data_nn_dummies_std[split_val:]
y_test = target[split_val:]
y_test = np.asarray(y_test).astype('float32').reshape((-1,1))


In [157]:
print(len(x_train), len(x_val) ,len(x_test),len(x_train)+len(x_val)+len(x_test) == len(data_nn_dummies_std))

29476 4912 14739 True


In [158]:
num_features = x_train.shape[1]
num_output = len(target.unique())
num_output


AttributeError: 'numpy.ndarray' object has no attribute 'unique'

In [159]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

# define the keras model
model = Sequential()
model.add(Dense(1250, input_dim=num_features, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(750, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(500, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(num_output, activation='sigmoid'))

In [160]:
model.compile(loss='CategoricalCrossentropy', optimizer='adam', metrics=['accuracy'])

# fit the keras model on the dataset
model.fit(x_train, y_train, epochs=10, batch_size=150, validation_data=(x_val,y_val))



ValueError: Data cardinality is ambiguous:
  x sizes: 29476
  y sizes: 16771844
Make sure all arrays contain the same number of samples.